In [ ]:
import pandas as pd
import tweepy
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

# Recopilación de datos de Twitter para usuarios legítimos
## Se utiliza el módulo Tweepy, se necesita: Consumer_KEY, Consumer_secret_KEY, Access_token, Access_Token_Secret

In [ ]:
consumer_key  = 'd9Ksoz6Wb1jD0mqbW8rjaSNb7'
consumer_secret = 'pHXnVSJeLbOxaYlbOR7BWFdDNhZSF6IzegZV87qUSUqy6Qe8qG'
access_token = '3648603434-dGRu1nHet22tdoYeqaAGoN8MyZrNw9oXZQvGZUD'
access_token_secret = 'PZ8pcQBCb5zVPLRQNVQZc3Yzi0rz1wPef6O7RO7gzcvOf'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

## Para usuarios genuinos, colecto una lista de 2500 usuarios seguidos por screen_name=@verified, esos usuarios están verificados y se guardan en un archivo .txt

In [ ]:
print('Nombre de usuarios seguidos por @verified')
friends = []
for friend in tweepy.Cursor(api.friends, screen_name = 'verified').items(2500):
    try:
        friends.append(friend.screen_name)
        print(friend.screen_name)
        time.sleep()
    except Exception as e:
        pass

with open("Data/Reales/verified-2500.txt", "w") as f:
     for item in friends:
        f.write("%s\n" % item)

## Recopilo 30 tweets de cada usuario verificado extraído de Twitter

In [ ]:
listaTweetsUsuariosLegitimos = []
fo = open("../Data/Reales/verified-2500.txt", "r")
f = fo.readlines()
fo.close()
dataset = map(lambda s: s.strip(),f)
counter = 0
try:
    for user in dataset:
        counter = counter + 1
        print(f'[{counter}] Obteniendo a lo sumo 30 tweets de {user}. ', end = '\n')
        try:
            for status in tweepy.Cursor(api.user_timeline, id = user, tweet_mode="extended").items(30):
                listaTweetsUsuariosLegitimos.append(status)
        except Exception as e:
            pass
except Exception as e:
    pass
print(len(listaTweetsUsuariosLegitimos))

In [ ]:
import urllib.parse

def process_http(string):
    url_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if s and n:
            url_count += 1
    return url_count

def process_hashtag(string):
    hashtag_count = 0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '#':
            hashtag_count += 1
    return hashtag_count

def process_mention(string):
    mention_count=0
    for i in string.split():
        s, n, p, pa, q, f = urllib.parse.urlparse(i)
        if i[:1] == '@':
            mention_count += 1
    return mention_count

def process_data(listaTweets):
    TweetID = [tweet.id for tweet in listaTweets]

    Data = pd.DataFrame(TweetID, columns = ['TweetID'])
    
    Data["TextData"] = [tweet.full_text for tweet in listaTweets]
    Data["TweetCreatedAt"] = [tweet.created_at for tweet in listaTweets]
    Data["RetweetCount"] = [tweet.retweet_count for tweet in listaTweets]
    Data["TweetFavouriteCount"] = [tweet.favorite_count for tweet in listaTweets]
    Data["TweetSource"] = [tweet.source for tweet in listaTweets]
    
    Data["UserID"] = [tweet.author.id for tweet in listaTweets]
    Data["UserScreenName"] = [tweet.author.screen_name for tweet in listaTweets]
    Data["UserName"] = [tweet.author.name for tweet in listaTweets]
    Data["UserCreatedAt"] = [tweet.author.created_at for tweet in listaTweets]
    Data["UserDescription"] = [tweet.author.description for tweet in listaTweets]
    Data["UserDescriptionLength"] = [len(tweet.author.description) for tweet in listaTweets]
    Data["UserFollowersCount"] = [tweet.author.followers_count for tweet in listaTweets]
    Data["UserFriendsCount"] = [tweet.author.friends_count for tweet in listaTweets]
    Data["UserLocation"] = [tweet.author.location for tweet in listaTweets]
    
    Data["HttpCount"] = [process_http(tweet.full_text) for tweet in listaTweets]
    Data["HashtagCount"] = [process_hashtag(tweet.full_text) for tweet in listaTweets]
    Data["MentionCount"] = [process_mention(tweet.full_text) for tweet in listaTweets]
    Data["TweetCount"] = [tweet.author.statuses_count for tweet in listaTweets]
    return Data

## De cada tweet, se extraen atributos útiles que se necesitan después para la clasificación

In [ ]:
Data = process_data(listaTweetsUsuariosLegitimos)
Data.shape

## Se guardan los datos en un archivo .csv

In [ ]:
import sys
Data.to_csv('../Data/Reales/Reales_data.csv', sep=',' , encoding='utf8', index=False)

## Para usuarios bots, colecto algunos usuarios de una lista proveída en botwiki-2019 obtenida en https://botometer.iuni.iu.edu/bot-repository/datasets.html

In [ ]:
import csv
tsv_file = open("../Data/Bots/botwiki-2019.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")
counter = 0
bots = []
for row in read_tsv:
    counter = counter + 1
    idBot = row[0]
    try:
        user = api.get_user(idBot)
        print(f'[{counter}] {user.screen_name}.', end = '\n')
        bots.append(user.screen_name)
    except tweepy.TweepError as e:
        counter = counter - 1
        mensajeError = e.args[0][0]['message']
        print(f'{idBot}: {mensajeError}', end = '\n')
        pass
tsv_file.close()
with open("../Data/Bots/botwiki-2019.txt", "w") as f:
     for item in bots:
        f.write("%s\n" % item)

## Recopilo 40 tweets de cada usuario bot extraído de botwiki-2019.tsv

In [ ]:
listaTweetsBots = []
fo = open("../Data/Bots/botwiki-2019.txt", "r")
f = fo.readlines()
fo.close()
dataset = map(lambda s: s.strip(),f)
counter = 0
try:
    for user in dataset:
        counter = counter + 1
        print(f'[{counter}] Obteniendo al menos 40 tweets de {user}. ', end = '\n')
        try:
            for status in tweepy.Cursor(api.user_timeline, id = user, tweet_mode="extended").items(40):
                listaTweetsBots.append(status)
        except Exception as e:
            pass
except Exception as e:
    pass
print(len(listaTweetsBots))

## De cada tweet, se extraen atributos útiles que se necesitan después para la clasificación

In [ ]:
Data = process_data(listaTweetsBots)
Data.shape

## Se guardan los datos en un archivo .csv

In [ ]:
import sys
# Saving data with item space separating
Data.to_csv('../Data/Bots/Bots_data.csv', sep=',' , encoding='utf8', index=False)

## Cargo los datos de los usuarios legítimos

In [ ]:
Total_leg_data = pd.read_csv('../Data/Reales/Reales_data.csv')
Total_leg_data.fillna(0, inplace=True)
Total_leg_data.shape

## Cargo los datos de los usuarios bots

In [ ]:
Total_bot_data = pd.read_csv("../Data/Bots/Bots_data.csv")
#Fill NAN values to 0
Total_bot_data.fillna(0, inplace=True)
Total_bot_data.shape

## Gráfico de barras que visualiza la ubicación proveniente de los tweets

In [ ]:
location_data = Total_leg_data['UserLocation'].value_counts()
location_data[2:15].plot(kind='bar', figsize=(14,7))

In [ ]:
location_data = Total_bot_data['UserLocation'].value_counts()
location_data[2:15].plot(kind='bar', figsize=(14,7))

# Agrego diferentes características al dataset al calcular:

# 1. AvgHashtag
# 2. AvgURLCount
# 3. AvgMention
# 4. AvgRetweet
# 5. AvgFavCount

In [ ]:
Total_leg_data.loc[:,"AvgHashtag"] = (Total_leg_data.groupby('UserID')["HashtagCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgURLCount"] = (Total_leg_data.groupby('UserID')["HttpCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgMention"] = (Total_leg_data.groupby('UserID')["MentionCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgRetweet"] = (Total_leg_data.groupby('UserID')["RetweetCount"].transform('sum'))/30
Total_leg_data.loc[:,"AvgFavCount"] = (Total_leg_data.groupby('UserID')["TweetFavouriteCount"].transform('sum'))/30

In [ ]:
Total_bot_data.loc[:,'AvgHashtag'] = (Total_bot_data.groupby('UserID')["HashtagCount"].transform('sum'))/40
Total_bot_data.loc[:,'AvgURLCount'] = (Total_bot_data.groupby('UserID')["HttpCount"].transform('sum'))/40
Total_bot_data.loc[:,'AvgMention'] = (Total_bot_data.groupby('UserID')["MentionCount"].transform('sum'))/40
Total_bot_data.loc[:,'AvgRetweet'] = (Total_bot_data.groupby('UserID')["RetweetCount"].transform('sum'))/40
Total_bot_data.loc[:,'AvgFavCount'] = (Total_bot_data.groupby('UserID')["TweetFavouriteCount"].transform('sum'))/40

## Selecciono solo columnas repetidas y elimino las filas repetidas

In [ ]:
unique_leg_row = Total_leg_data[["UserID", "UserScreenName", "UserName", "UserCreatedAt", "UserDescription", "UserDescriptionLength","UserFollowersCount", "UserFriendsCount", "UserLocation", "AvgHashtag", "AvgURLCount", "AvgMention", "AvgRetweet", "AvgFavCount", "TweetCount"]]
leg_data = unique_leg_row.drop_duplicates()
leg_data.info()

In [ ]:
unique_spam_row = Total_bot_data[["UserID", "UserScreenName", "UserName", "UserCreatedAt", "UserDescription", "UserDescriptionLength","UserFollowersCount", "UserFriendsCount", "UserLocation", "AvgHashtag", "AvgURLCount", "AvgMention", "AvgRetweet", "AvgFavCount", "TweetCount"]]
bot_data = unique_spam_row.drop_duplicates()
bot_data.info()

In [ ]:
import datetime
CurrentDate = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')
leg_data.loc[:, "CurrentDate"] = CurrentDate
leg_data.loc[:, "bot"] = 0
leg_data.tail()

In [ ]:
CurrentDate = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')
bot_data.loc[:, "CurrentDate"] = CurrentDate
bot_data.loc[:, "bot"] = 1
bot_data.tail()

## Fusiono los datos de los usuarios legítimos y bots.

In [ ]:
frames = [leg_data, bot_data]
Total_data = pd.concat(frames, axis=0, sort=False)
Total_data.info()

## Se guardan los datos en un archivo .csv

In [ ]:
Total_data.reset_index()
Total_data.to_csv('../Data/Dataset.csv', sep=',', encoding='utf8', index = False)

# Cargo los datos totales (legítimos y bots)
# Desde acá comenzará el aprendizaje automatizado (machine learning), prueba con un clasificador Random Forest

In [ ]:
Total_data = pd.read_csv('../Data/Dataset.csv')
Total_data.fillna(0, inplace=True)
Total_data.shape

# Agrego la característica Reputación

In [ ]:
Total_data.loc[:,"Reputation"]=Total_data["AvgRetweet"]/(Total_data["UserFollowersCount"])
Total_data["Reputation"].describe()

## Agrego la característica AgeOfAccount

In [ ]:
data = Total_data
data["CurrentDate"] = pd.to_datetime(data["CurrentDate"])
data["UserCreatedAt"] = pd.to_datetime(data["UserCreatedAt"])
data['AgeOfAccount'] = (data['CurrentDate'] - data['UserCreatedAt'])/np.timedelta64(1, 'D')
cols = ['AgeOfAccount']
data[cols] = data[cols].mask(data[cols]<0)
data.AgeOfAccount.describe()

## Agrego característica tweet por día

In [ ]:
data1 = data
data1.loc[:, "TweetPerDay"] = data1["TweetCount"]/data1["AgeOfAccount"]
data1["TweetPerDay"].describe()

## Agrego característica tweet por seguidor

In [ ]:
data1.loc[:,"TweetPerFollower"] = data1["TweetCount"]/data1["UserFollowersCount"]

In [ ]:
#to remove unwanted data
data1.TweetPerFollower=data1.TweetPerFollower.round(2).fillna(0)
data1 = data1[np.isfinite(data1['TweetPerFollower'])]
data1["TweetPerFollower"].tail(3)

## Agrego la característica Age of Account/Número de seguidos
## Hipótesis: es muy bajo para spammers/bots y muy alto para usuarios legítimos

In [ ]:
data1.loc[:,"AgeByFollowing"] = data1["AgeOfAccount"]/data1["UserFriendsCount"]
data1 = data1[np.isfinite(data1['AgeByFollowing'])]
data1[['AgeByFollowing']] = data1[['AgeByFollowing']].astype(float)
data1["AgeByFollowing"].describe()

## Agrego características que indican si screen_name, name o description contienen alguna palabra relacionada a bot

In [ ]:
data1.loc[:,'screen_name_binary'] = (data1.UserScreenName.str.contains("Bot|bot|b0t|B0t|BOT", case=False)==True)
data1['name_binary'] = (data1.UserName.str.contains("Bot|bot|b0t|B0t|BOT", case=False)==True)
data1['description_binary'] = (data1.UserDescription.str.contains("Bot|bot|b0t|B0t|BOT", case=False)==True)

In [ ]:
data1[['screen_name_binary']] = data1[['screen_name_binary']].astype(int)
data1[['name_binary']] = data1[['name_binary']].astype(int)
data1[['description_binary']] = data1[['description_binary']].astype(int)
data1.info()

In [ ]:
Total_data.columns

## Selecciono las características para la predicción y el target

In [ ]:
X = data1[['Reputation', 'AvgHashtag', 'AvgRetweet', 'UserFollowersCount','UserFriendsCount', 'AvgFavCount', 'AvgMention', 'AvgURLCount', 'TweetCount', 'AgeOfAccount', 'TweetPerDay', 'TweetPerFollower', 'AgeByFollowing', 'screen_name_binary', 'name_binary', 'description_binary']]
y = data1["bot"]
print(X)

# Implemento el clasificador Random Forest

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=7, max_depth=7, min_samples_split=5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

rf = rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

print("Trainig Accuracy: %.5f" %accuracy_score(y_train, y_pred_train))
print("Test Accuracy: %.5f" %accuracy_score(y_test, y_pred_test))
print("Classifier performance report: ")
print(classification_report(y_test, y_pred_test))

In [ ]:
sns.set_style("whitegrid", {'axes.grid' : False})

scores_train = rf.predict_proba(X_train)
scores_test = rf.predict_proba(X_test)

y_scores_train = []
y_scores_test = []
for i in range(len(scores_train)):
    y_scores_train.append(scores_train[i][1])

for i in range(len(scores_test)):
    y_scores_test.append(scores_test[i][1])
    
fpr_rf_train, tpr_rf_train, _ = roc_curve(y_train, y_scores_train, pos_label=1)
fpr_rf_test, tpr_rf_test, _ = roc_curve(y_test, y_scores_test, pos_label=1)
fig=plt.figure(figsize=(8, 7), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(fpr_rf_train, tpr_rf_train, color='darkblue', label='Train AUC: %5f' %auc(fpr_rf_train, tpr_rf_train))
plt.plot(fpr_rf_test, tpr_rf_test, color='red', ls='--', label='Test AUC: %5f' %auc(fpr_rf_test, tpr_rf_test))
plt.title("Random ForestROC Curve")
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.legend(loc='lower right')

# Prueba de predicción

In [ ]:
Predict_Data = []
#user = 'afa' // resultado 0
user = 'RecuerdameBot'
counter = 0
try:
    counter = counter + 1
    print(f'[{counter}] Obteniendo 30 tweets de {user}. ', end = '\n')
    try:
        for status in tweepy.Cursor(api.user_timeline, id = user, tweet_mode="extended").items(30):
                Predict_Data.append(status)
    except Exception as e:
        pass
except Exception as e:
    pass
print(len(Predict_Data))

In [ ]:
Data = process_data(Predict_Data)
Data.shape

In [ ]:
Data.loc[:,"AvgHashtag"] = (Data.groupby('UserID')["HashtagCount"].transform('sum'))/30
Data.loc[:,"AvgURLCount"] = (Data.groupby('UserID')["HttpCount"].transform('sum'))/30
Data.loc[:,"AvgMention"] = (Data.groupby('UserID')["MentionCount"].transform('sum'))/30
Data.loc[:,"AvgRetweet"] = (Data.groupby('UserID')["RetweetCount"].transform('sum'))/30
Data.loc[:,"AvgFavCount"] = (Data.groupby('UserID')["TweetFavouriteCount"].transform('sum'))/30

In [ ]:
unique_spam_row = Data[["UserID", "UserScreenName", "UserName", "UserCreatedAt", "UserDescription", "UserDescriptionLength","UserFollowersCount", "UserFriendsCount", "UserLocation", "AvgHashtag", "AvgURLCount", "AvgMention", "AvgRetweet", "AvgFavCount", "TweetCount"]]
predict_data = unique_spam_row.drop_duplicates()
predict_data.info()

In [ ]:
print(predict_data)

In [ ]:
import datetime
Current_Time = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')
predict_data.loc[:, "CurrentDate"]=Current_Time
Predict_data = predict_data

In [ ]:
Predict_data.loc[:,"Reputation"]=Predict_data["AvgRetweet"]/(Predict_data["UserFollowersCount"])
Predict_data["Reputation"].describe()

In [ ]:
data = Predict_data
data["CurrentDate"] = pd.to_datetime(data["CurrentDate"])
data["UserCreatedAt"] = pd.to_datetime(data["UserCreatedAt"])
data['AgeOfAccount'] = (data['CurrentDate'] - data['UserCreatedAt'])/np.timedelta64(1, 'D')
cols = ['AgeOfAccount']
data[cols] = data[cols].mask(data[cols]<0)
data.AgeOfAccount.describe()

In [ ]:
data1 = data
data1.loc[:, "TweetPerDay"] = data1["TweetCount"]/data1["AgeOfAccount"]
data1["TweetPerDay"].describe()

In [ ]:
data1.loc[:,"TweetPerFollower"] = data1["TweetCount"]/data1["UserFollowersCount"]

In [ ]:
#to remove unwanted data
data1.TweetPerFollower=data1.TweetPerFollower.round(2).fillna(0)
data1 = data1[np.isfinite(data1['TweetPerFollower'])]
data1["TweetPerFollower"].tail(3)

In [ ]:
if(data1["UserFriendsCount"].item() > 0):
    data1.loc[:,"AgeByFollowing"] = data1["AgeOfAccount"]/data1["UserFriendsCount"]
    data1 = data1[np.isfinite(data1['AgeByFollowing'])]
    data1[['AgeByFollowing']] = data1[['AgeByFollowing']].astype(float)
else:
    data1.loc[:,"AgeByFollowing"] = 0
data1["AgeByFollowing"].describe()

In [ ]:
data1.loc[:,'screen_name_binary'] = (data1.UserScreenName.str.contains("Bot|bot|b0t|B0t|BOT", case=False)==True)
data1['name_binary'] = (data1.UserName.str.contains("Bot|bot|b0t|B0t|BOT", case=False)==True)
data1['description_binary'] = (data1.UserDescription.str.contains("Bot|bot|b0t|B0t|BOT", case=False)==True)
data1[['screen_name_binary']] = data1[['screen_name_binary']].astype(int)
data1[['name_binary']] = data1[['name_binary']].astype(int)
data1[['description_binary']] = data1[['description_binary']].astype(int)
data1.info()

In [ ]:
X = data1[['Reputation', 'AvgHashtag', 'AvgRetweet', 'UserFollowersCount','UserFriendsCount', 'AvgFavCount', 'AvgMention', 'AvgURLCount', 'TweetCount', 'AgeOfAccount', 'TweetPerDay', 'TweetPerFollower', 'AgeByFollowing', 'screen_name_binary', 'name_binary', 'description_binary']]
print(X)

In [ ]:
prediction = rf.predict(X)

In [ ]:
print(prediction)